In [66]:
import torch

In [67]:
names = open("names.txt", "r").read().splitlines()

In [68]:
# collect a list of all the letters
name_set = set()
for name in names:
    for c in name:
        name_set.add(c)

char_list = list(name_set)
char_list.append(".")
sorted_letter = sorted(char_list)

# create a dictionary that contains the mapping of the index to the character
stoi = {s:i for i, s in enumerate(sorted_letter)}
itos = {i:s for s, i in stoi.items()}

In [75]:
# set out the letter counting
# i do not get how this can be visualised though...
# counts = torch.zeros((27, 27, 27))

# input this will take a tuple
x = []

# output
y = []

for word in names:
    formatted_word = "." + word + "."
    for c1, c2, c3 in zip(formatted_word, formatted_word[1:], formatted_word[2:]):
        x.append((stoi[c1], stoi[c2]))
        y.append(stoi[c3])

x = torch.tensor(x)
y = torch.tensor(y)

x.shape[0]

196113

In [76]:
input_tensors = []

# try making my own variation of ohe that can represent 2 inputs
for count, v in enumerate(x):
    base = torch.zeros((27, 27))
    # everything that starts with '.' will be in the first row. anything that ends with '.' will be in the first column
    base[v[0], v[1]] += 1
    # print(base.shape)
    input_tensors.append(base)

xenc = torch.stack(input_tensors, dim=0)
xenc.shape

# reshape the tensor for multiplication -> 3 states the number of rows, while -1 means that the column dimension can be inferred after calculation
xenc = xenc.view(x.shape[0], -1)
xenc.shape

torch.Size([196113, 729])

In [41]:
xenc

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])

In [77]:
# now that all the data has been prepared, it is time to make the neural network

# start by creating weights from the seed
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)
W.shape

torch.Size([729, 27])

In [78]:
for i in range(200):
    logits = xenc @ W
    exp = logits.exp()
    sum = exp.sum(dim=1, keepdim=True)
    probs = exp / sum

    loss = -probs[torch.arange(xenc.shape[0]), y[0]].log().mean()
    print(f"{i}: {loss}")

    W.grad = None
    loss.backward()

    W.data += -10 * W.grad

0: 3.7925939559936523
1: 3.712592124938965
2: 3.6333956718444824
3: 3.5550966262817383
4: 3.477797269821167
5: 3.4016079902648926
6: 3.3266489505767822
7: 3.2530415058135986
8: 3.1809046268463135
9: 3.110346794128418
10: 3.0414631366729736
11: 2.9743270874023438
12: 2.9089927673339844
13: 2.8454933166503906
14: 2.783843994140625
15: 2.724045991897583
16: 2.6660871505737305
17: 2.609947681427002
18: 2.555598258972168
19: 2.503004550933838
20: 2.452125072479248
21: 2.402916669845581
22: 2.355332136154175
23: 2.3093228340148926
24: 2.2648396492004395
25: 2.2218334674835205
26: 2.1802544593811035
27: 2.1400551795959473
28: 2.1011877059936523
29: 2.063605785369873
30: 2.027263641357422
31: 1.9921172857284546
32: 1.9581228494644165
33: 1.9252378940582275
34: 1.8934203386306763
35: 1.862629771232605
36: 1.8328256607055664
37: 1.803969144821167
38: 1.7760224342346191
39: 1.7489479780197144
40: 1.722710132598877
41: 1.6972737312316895
42: 1.6726058721542358
43: 1.6486736536026
44: 1.62544643878

In [85]:
# now we cna try to sample from the trigram model. LOL

import random
# random.seed(2147483647)

g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    # for the first, choose the second letter as a random letter
    first_index = 0
    second_index = random.randint(1, 26)
    output = [itos[second_index]]

    while True:
        # create a one hot encoding for the first two letters
        base = torch.zeros((27, 27))
        base[first_index, second_index] += 1
        # squash it into a one-hot encoding that can be fed into the NN
        xenc = base.view(1, -1) # this would probably be 1, 729

        logits = xenc @ W
        probs = logits.exp()
        P = probs / probs.sum(dim=1, keepdim=True)
        output_index = torch.multinomial(P, num_samples=1, replacement=True, generator=g).item()

        output.append(itos[output_index])
        first_index = second_index
        second_index = output_index

        if output_index == 0:
            break

    print("".join(output))
    

vmexzdfzjglkurmimczkwyhhmvlzimmtnammmlbfvk.
mmammammwmxqpuhptthmmimmwxezgzjedmnvugkwptedmwekjemmmmsammy.
amkxvjrnfrqtbspmhwcjdemmtrmxuow.
wmsfxxblgjxlhgfiwuipgnammwfdnimcwzktsdemm.
zmommt.


#########
Below are some of the intermediate working cells
########